# Cálculo en Arrays de NumPy: funciones universales

## La lentitud de los bucles ¶ 

La relativa lentitud de Python generalmente se manifiesta en situaciones en las que se repiten muchas operaciones pequeñas

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

Si medimos el tiempo de ejecución de este código para una entrada grande, vemos que esta operación es muy lenta.

In [6]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array) 

4.8 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


¡Toma varios segundos calcular estos millones de operaciones y almacenar el resultado!

Resulta que el cuello de botella es causado cada vez que se calcula el recíproco, Python primero examina el tipo de objeto y realiza una búsqueda dinámica de la función correcta que se utilizará para ese tipo. 

## Presentamos UFuncs ¶ 

Esto se conoce como operación vectorizada. Se puede lograr simplemente realizando una operación en el array.

In [7]:
print(compute_reciprocals(values))
#Operación Vectorizada 
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


Al observar el tiempo de ejecución de nuestro big_array, vemos que completa órdenes de magnitud más rápido que el ciclo de Python.

In [5]:
%timeit (1.0 / big_array)

4.18 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



Las operaciones vectorizadas en NumPy se implementan a través ufuncs , cuyo objetivo principal es ejecutar rápidamente operaciones repetidas en valores en array de NumPy. 

Los ufuncs son extremadamente flexibles: antes vimos una operación entre un escalar y una matriz, pero también podemos operar entre dos matrices:

In [8]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])


Y las operaciones de ufunc no se limitan a matrices unidimensionales, también pueden actuar en matrices multidimensionales:

In [9]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

Los cálculos que usan vectorización a través de ufuncs son casi siempre más eficientes que su contraparte implementada usando bucles de Python, especialmente a medida que los arrays aumentan de tamaño.

Cada vez que vea un bucle de este tipo en una secuencia de comandos de Python, debe considerar si se puede reemplazar con una expresión vectorizada.



## Explorando las UFuncs de NumPy

Los ufuncs existen en dos versiones: 
    
    Unarios->  Operan en una sola entrada.
    Binarios-> Operan en dos entradas.


### Aritmética de arreglos

Los ufuncs de NumPy se sienten muy naturales de usar porque hacen uso de los operadores aritméticos nativos de Python. Se pueden usar las +, -, * y / estándar:

In [11]:
x = np.arange(4)
print(f"x     = {x}")
print(f"x + 5 ={x+5}")
print(f"x - 5 ={x-5}")
print(f"x * 2 ={x*2}")
print(f"x / 2 ={x/2}")
print(f"x // 2 ={x // 2}")  # floor division

x     = [0 1 2 3]
x + 5 =[5 6 7 8]
x - 5 =[-5 -4 -3 -2]
x * 2 =[0 2 4 6]
x / 2 =[0.  0.5 1.  1.5]
x // 2 =[0 0 1 1]


In [15]:
print(f"-x   ={-x} ")
print(f"x**2 ={x**2} ")
print(f"x%2  ={x%2} ")


-x   =[ 0 -1 -2 -3] 
x**2 =[0 1 4 9] 
x%2  =[0 1 0 1] 



Además, se pueden encadenar como se desee y se respeta el orden estándar de las operaciones:

In [19]:
-(2*x + 1) ** 2

array([ -1,  -9, -25, -49])

Cada una de estas operaciones aritméticas son simplemente envoltorios convenientes alrededor de funciones específicas integradas en NumPy; por ejemplo

El operador "+" es un envoltorio para la función add:

In [20]:
np.add(x, 2)

array([2, 3, 4, 5])

La siguiente tabla enumera los operadores aritméticos implementados en NumPy:


        Operator 	Equivalent              ufunc

        +               np.add                  Addition (1+1 = 2)
        
        - 	        np.subtract 	        Subtraction (3-2 = 1)
        
        - 	        np.negative 	        Unary negation (-2)

        * 	        np.multiply 	        Multiplication (2*3 = 6)

        / 	        np.divide 	        Division (3/2 = 1.5)

        // 	        np.floor_divide 	Floor division (3//2 = 1)

        ** 	        np.power 	        Exponentiation (2**3 = 8)

        % 	        np.mod 	                Modulus/remainder (9%4 = 1)
        

### Valor absoluto

Así como NumPy comprende los operadores aritméticos integrados de Python, también comprende la función de valor absoluto integrada de Python: 

In [23]:
x = np.array([-2, -1, 0, 1, -7])
abs(x)

array([2, 1, 0, 1, 7])



El NumPy ufunc correspondiente es 'np.absolute', disponible bajo el alias 'np.abs'


In [25]:
print(f'{np.absolute(x)}')

print(f'{np.abs(x)}')

[2 1 0 1 7]
[2 1 0 1 7]




Este ufunc también puede manejar datos complejos, en los que el valor absoluto equivale a la magnitud.


In [26]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Funciones trigonométricas

NumPy proporciona una gran cantidad de ufuncs útiles, y algunos de los más útiles para el científico de datos son las funciones trigonométricas. 

In [28]:
#Definimos un array de ángulos: 

theta = np.linspace(0, np.pi, 3)
theta

array([0.        , 1.57079633, 3.14159265])

Ahora podemos calcular algunas funciones trigonométricas sobre estos valores:

In [29]:
print(f"theta      = {theta}")
print(f"sin(theta) = {np.sin(theta)}")
print(f"cos(theta) = {np.cos(theta)}")
print(f"tan(theta) = {np.tan(theta)}")

theta      = [0.         1.57079633 3.14159265]
sin(theta) = [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) = [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) = [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]




Los valores se calculan dentro de la precisión de la máquina, por lo que los valores que deberían ser cero no siempre llegan exactamente a cero. 

Las funciones trigonométricas inversas también están disponibles:


In [30]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))


x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponents and logarithms

Otro tipo común de operación disponible en un NumPy ufunc son las exponenciales.

In [36]:
x = [1, 2, 3]

print(f"x     = {x}")
print(f"e^x   = {np.exp(x)}", )
print(f"2^x   = {np.exp2(x)}")
print(f"3^x   = {np.power(3,x)}", )

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]




También están disponibles los inversos de las exponenciales, los logaritmos.

Lo básico 'np.log' da el logaritmo natural; si prefiere calcular el logaritmo en base 2 o el logaritmo en base 10, también están disponibles:


In [34]:
x = [1, 2, 4, 10]
print(f"x        = {x}")
print(f"ln(x)    = {np.log(x)}")
print(f"log2(x)  = {np.log2(x)}")
print(f"log10(x) = {np.log10(x)}")

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


También hay algunas versiones especializadas que son útiles para mantener la precisión con una entrada muy pequeña-

In [37]:
x = [0, 0.001, 0.01, 0.1]
print(f"exp(x) - 1 = {np.expm1(x)}")
print(f"log(1 + x) = {np.log1p(x)}")


exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


Cuándo x es muy pequeño, estas funciones dan valores más precisos que si el crudo np.log o np.exp iban a ser utilizados.

### ufuncs especializados

NumPy tiene muchos más ufuncs disponibles, incluidas funciones:
+ Trigonométricas hiperbólicas
+ Aritmética bit a bit
+ Operadores de comparación
+ Conversiones de radianes a grados
+ Redondeo y resto
+ y mucho más.

 Un vistazo a la documentación de NumPy revela muchas funcionalidades interesantes. 

Otra fuente excelente para ufuncs más especializados y oscuros es el submódulo scipy.special. 
Si desea calcular alguna función matemática oscura en sus datos, es probable que esté implementada en scipy.special.

Hay demasiadas funciones para enumerarlas todas, pero el siguiente fragmento muestra un par que podría surgir en un contexto de estadísticas: 

In [38]:
from scipy import special

In [39]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [41]:
# Función de error (integral de gaussiana)
# su complemento y su inversa

x = np.array([0, 0.3, 0.7, 1.0])
print(f"erf(x)  = {special.erf(x)}")
print(f"erfc(x) = {special.erfc(x)}")
print(f"erfinv(x) = {special.erfinv(x)}")

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


Hay muchos, muchos más ufuncs disponibles tanto en NumPy como en scipy.special.

Debido a que la documentación de estos paquetes está disponible en línea, una búsqueda en la web similar a "gamma function python" generalmente encontrará la información relevante.

## Funciones avanzadas de Ufunc ¶ 

Muchos usuarios de NumPy hacen uso de ufuncs sin siquiera aprender su conjunto completo de funciones.
Describiremos algunas características especializadas de ufuncs aquí. 

### Especificando salida

Para cálculos grandes, a veces es útil poder especificar el array donde se almacenará el resultado del cálculo. En lugar de crear una matriz temporal, esto se puede usar para escribir los resultados de los cálculos directamente en la ubicación de la memoria donde le gustaría que estuvieran. Para todos los ufuncs, esto se puede hacer usando el out argumento de la función: 

In [44]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]




Esto incluso se puede usar con vistas de arrays. 

Por ejemplo, podemos escribir los resultados de un cálculo en cualquier otro elemento de una arrays específica:


In [43]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)


[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]




Si en cambio hubiéramos escrito 'y[::2]=2** x' , esto habría resultado en la creación de una matriz temporal para contener los resultados de '2**x', seguido de una segunda operación copiando esos valores en el 'y' formación. Esto no hace una gran diferencia para un cálculo tan pequeño, pero para arreglos muy grandes, el ahorro de memoria por el uso cuidadoso del 'out' argumento puede ser significativo.


### Agregados

Para ufuncs binarios, hay algunos agregados interesantes que se pueden calcular directamente desde el objeto. Por ejemplo, si quisiéramos reducir una matriz con una operación particular, podemos usar el método 'reduce' de cualquier ufunc. 

Una reducción aplica repetidamente una operación determinada a los elementos de una matriz hasta que solo queda un único resultado. 

Por ejemplo, llamando 'reduce' sobre el 'add' ufunc devuelve la suma de todos los elementos de la matriz.

In [57]:
x = np.arange(1, 11)
np.add.reduce(x)  # Equivalente a x.sum() pero es ufunc

55

Del mismo modo, llamar 'reduce' sobre el 'multiply' ufunc da como resultado el producto de todos los elementos de la matriz:

In [58]:
np.multiply.reduce(x) # Equivalente a 10! en arange

3628800

Si deseamos almacenar todos los resultados intermedios del cálculo, podemos usar en su lugar 'accumulate'

In [59]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15, 21, 28, 36, 45, 55])

In [61]:
np.multiply.accumulate(x)

array([      1,       2,       6,      24,     120,     720,    5040,
         40320,  362880, 3628800])

Tenga en cuenta que para estos casos particulares, hay funciones NumPy dedicadas para calcular los resultados ( np.sum, np.prod, np.cumsum, np.cumprod), que exploraremos en Agregaciones: Mín., Máx. y todo lo intermedio .

### Productos exteriores 

Finalmente, cualquier ufunc puede calcular la salida de todos los pares de dos entradas diferentes usando el método 'outer'.

 Esto le permite, en una línea, hacer cosas como crear una tabla de multiplicar.

In [62]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])



Los métodos 'ufunc.at' y 'ufunc.reduceat', también son muy útiles.

Otra característica extremadamente útil de ufuncs es la capacidad de operar entre matrices de diferentes tamaños y formas, un conjunto de operaciones conocido como transmisión . 
